<a href="https://colab.research.google.com/github/kato1329/CATech/blob/main/%E5%8C%BB%E7%99%82%E8%AB%96%E6%96%87%E4%BB%95%E5%88%86%E3%81%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [2]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.functional as F
from torch.utils.data import Dataset,DataLoader
from transformers import AutoModel
from transformers import AutoTokenizer

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
folder_path = './drive/MyDrive/comp_siwake/'
train_path = folder_path + 'train.csv'
test_path = folder_path + 'test.csv'

In [5]:
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [6]:
###abstract使いづらいー
train_df.isnull().sum()

id              0
title           0
abstract     4390
judgement       0
dtype: int64

In [7]:
###titleだけ使おうかな
train_df['title'].head()

0    One-year age changes in MRI brain volumes in o...
1    Supportive CSF biomarker evidence to enhance t...
2    Occurrence of basal ganglia germ cell tumors w...
3    New developments in diagnosis and therapy of C...
4    Prolonged shedding of SARS-CoV-2 in an elderly...
Name: title, dtype: object

In [8]:
train_x = pd.DataFrame()
train_x['text'] = train_df['title']
train_x['label'] = train_df['judgement']

In [9]:
train_x.head()

,text,label
0,One-year age changes in MRI brain volumes in o...,0
1,Supportive CSF biomarker evidence to enhance t...,0
2,Occurrence of basal ganglia germ cell tumors w...,0
3,New developments in diagnosis and therapy of C...,0
4,Prolonged shedding of SARS-CoV-2 in an elderly...,0


In [10]:
from sklearn.model_selection import train_test_split

data = train_x[["text","label"]]
train_data,test_data = train_test_split(data,test_size = 0.2)

In [ ]:
!pip install fugashi
!pip install ipadic

In [12]:
type(train_data)

pandas.core.frame.DataFrame

In [13]:
checkpoint = 'cl-tohoku/bert-base-japanese-whole-word-masking'
MAX_LEN = 256
tokenizer =AutoTokenizer.from_pretrained(checkpoint)
class MyDataset(Dataset):
  def __init__(self,data,tokenizer):
    self.data = pd.DataFrame(data)
    self.tokenizer = AutoTokenizer.from_pretrained(checkpoint)
  def __len__(self):
    return len(self.data)
  def __getitem__(self,idx):
    text = self.data["text"].tolist()[idx]
    label = self.data["label"].tolist()[idx]
    label = torch.tensor(label,dtype=torch.int64)

    encodings = self.tokenizer(
        text,padding = "max_length",
        max_length = MAX_LEN,
        truncation = True
    )

    input_ids = torch.tensor(
        encodings["input_ids"],dtype=torch.long
    )

    attention_mask = torch.tensor(
        encodings["attention_mask"],dtype=torch.long
    )

    token_type_ids = torch.tensor(
        encodings["token_type_ids"],dtype=torch.long
    )

    return input_ids,attention_mask,token_type_ids,label


In [14]:
train_data = MyDataset(train_data,tokenizer)
test_data = MyDataset(test_data,tokenizer)

train_loader = DataLoader(train_data,batch_size=16,shuffle=True,drop_last=True)
test_loader = DataLoader(test_data,batch_size=16,shuffle=True,drop_last=True)

In [15]:
###train_loaderの形状が知りたいので一つ取り出して確かめてみる
tmp = iter(train_loader)
input_ids = torch.Tensor()
for batch in tmp:
  input_ids,attention_mask,token_type_ids,label = batch
  break

In [24]:
class BERTclassification(nn.Module):
  def __init__(self):
    super().__init__()
    self.bert = AutoModel.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
    self.linear1 = nn.Linear(in_features=768,out_features=256)
    self.linear2 = nn.Linear(256,64)
    self.linear3 = nn.Linear(64,2)
    self.relu = nn.ReLU()
    self.softmax = nn.Softmax(dim=1)
  def forward(self,input_ids,attention_mask,token_type_ids):
    x = self.bert(
        input_ids = input_ids,
        attention_mask = attention_mask,
        token_type_ids = token_type_ids
    )
    x = x.pooler_output
    x = self.relu(self.linear1(x))
    x = self.relu(self.linear2(x))
    x = self.linear3(x)
    output = self.softmax(x)
    return output

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else'cpu')
model = BERTclassification().to(device)
optimizer = optim.Adam(model.parameters(),lr = 1e-5)
criterion = nn.CrossEntropyLoss()
epoch =10

In [26]:
from tqdm import tqdm

In [27]:
for i in range(epoch):
  print('---------------------')
  print("Epoch:{}/{}".format(i+1,epoch))

  train_loss = 0
  train_accuracy = 0
  test_loss = 0
  test_accuracy = 0

  model.train()

  for batch in tqdm(train_loader):
    optimizer.zero_grad()
    input_ids = batch[0].to(device)
    attention_mask = batch[1].to(device)
    token_type_ids = batch[2].to(device)

    label = batch[3].to(device)
    pred = model(input_ids,attention_mask,token_type_ids)
    loss = criterion(pred,label)
    loss.backward()
    optimizer.step()
    train_loss += loss.item()
    pred_label = torch.max(pred,1)[1]
    train_accuracy += torch.sum(pred_label.view(-1,16)==label).item()/len(label)
  train_loss = train_loss/len(train_loader)
  train_accuracy = train_accuracy/len(train_loader)
  print("train_loss = {}".format(train_loss))
  print("train_acc = {}".format(train_accuracy))
  model.eval()
  with torch.no_grad():
    for batch in tqdm(test_loader):
      input_ids = batch[0].to(device)
      attention_mask = batch[1].to(device)
      token_type_ids = batch[2].to(device)

      label = batch[3].to(device)
      pred = model(input_ids,attention_mask,token_type_ids)
      loss = criterion(pred,label)
      test_loss += loss.item()
      pred_label = torch.max(pred,1)[1]
      test_accuracy += torch.sum(pred_label.view(-1,16)==label).item()/len(label)
  test_loss = test_loss/len(test_loader)
  test_accuracy = test_accuracy/len(test_loader)
  print("test_loss = {}".format(test_loss))
  print("test_acc = {}".format(test_accuracy))

100%|██████████| 1357/1357 [15:41<00:00,  1.44it/s]


train_loss = 0.33665974676477267
train_acc = 0.9766028002947679


100%|██████████| 339/339 [01:22<00:00,  4.09it/s]


test_loss = 0.33612346543674976
test_acc = 0.9771386430678466
---------------------
Epoch:7/10


100%|██████████| 1357/1357 [15:41<00:00,  1.44it/s]


...

test_loss = 0.3361231966004611
test_acc = 0.9771386430678466
---------------------
Epoch:10/10


100%|██████████| 1357/1357 [15:40<00:00,  1.44it/s]


train_loss = 0.33665901169938556
train_acc = 0.9766028002947679


100%|██████████| 339/339 [01:23<00:00,  4.08it/s]

test_loss = 0.33612316495197714
test_acc = 0.9771386430678466
